In [1]:
from data_loader import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from keras.models import Model
from keras.layers import Dense, Embedding, Input, LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import roc_auc_score

Using TensorFlow backend.


In [2]:
train = load_train_data("data/train.csv")

In [3]:
test_ids, test_texts, test_labels = load_test_data("data/test.csv", "data/test_labels.csv", False)

/Users/Zry/Desktop/2019Spring/MachineLearning/Project/data_loader.py:27: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  return data_frame['id'].tolist(), data_frame['comment_text'].tolist(), label_frame.iloc[:, 1:].as_matrix()


In [4]:
train_labels = train[0].iloc[:,2:]

In [5]:
dim = 150
vol_size = 20000
emb_size = 128

In [6]:
X = train[0].comment_text
X_val = train[1].comment_text
Y = train[0][["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
Y_val = train[1][["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

In [7]:
tok = Tokenizer(num_words = vol_size)
tok.fit_on_texts(list(X))
tok.fit_on_texts(list(X_val))

In [8]:
X = tok.texts_to_sequences(X)
X_val = tok.texts_to_sequences(X_val)
X_test = tok.texts_to_sequences(test_texts)
X = sequence.pad_sequences(X, maxlen = dim)
X_val = sequence.pad_sequences(X_val, maxlen = dim)
X_test = sequence.pad_sequences(X_test, maxlen = dim)

In [ ]:
def lstm_model(layer_num = 1):
    _in = Input(shape = (dim, ))
    layer = Embedding(vol_size, emb_size)(_in)
    for i in range(layer_num):
        layer = Bidirectional(LSTM(50, return_sequences = True, recurrent_dropout = 0.15))(layer)
    layer = GlobalMaxPool1D()(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(50, activation = 'relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(6, activation = 'sigmoid')(layer)
    model = Model(inputs = _in, outputs = layer)
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    return model

In [ ]:
model = lstm_model(layer_num = 2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
stop = EarlyStopping(monitor = 'val_loss', patience = 1)

In [ ]:
res = model.fit(X, Y, batch_size = 512, epochs = 2, validation_data = (X_val, Y_val), callbacks = [stop])

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 143645 samples, validate on 15926 samples
Epoch 1/2
  3072/143645 [..............................] - ETA: 26:44 - loss: 0.6609 - acc: 0.8908

In [ ]:
Y_test = model.predict(X_test)

In [ ]:
sum = 0
for i in range(6):
    score = roc_auc_score(test_labels[:,i], Y_test[:,i])
    sum += score

In [ ]:
print (sum / 6)